In [14]:
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D 
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D 
from keras.optimizers import RMSprop, Adam
from keras.initializers import glorot_uniform

import os
from resnet_builder import resnet
import numpy as np
import pandas as pd

In [2]:
ROOT_DIR = os.path.abspath("../../data")
ROOT_DIR
TRAIN_DIR = os.path.join(ROOT_DIR, "train")
TRAIN_DIR
TEST_DIR = os.path.join(ROOT_DIR,"test")

In [15]:
IMG_DIR = os.path.join(ROOT_DIR,'img')

In [4]:
# Image Generator
train_datagen = ImageDataGenerator(rotation_range=20,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True, )
test_datagen = ImageDataGenerator(rescale=1./255)

# Misc Information
num_classes = 5
batch_size = 32


In [13]:
aug_gen = ImageDataGenerator(rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest'
                            )

In [39]:
x = aug_gen.flow_from_directory(IMG_DIR, target_size=(64,64),
                               batch_size=32, class_mode='categorical', 
                               save_to_dir=IMG_DIR)

Found 18 images belonging to 4 classes.


['daisy\\1b212edb4f26f7f9008bbf761568c751.jpg',
 'daisy\\1c28aecc1e573ca00b7f844de6d01d0c.jpg',
 'daisy\\1d1d51710e5f7412ac44a822842351f5.jpg',
 'daisy\\1d7433955a4559ea8a0437d86821a201.jpg',
 'rose\\001e16488b7ee3ee99cef0e8cde1ad95.jpg',
 'rose\\1a73115f72ec21595fa7f5e8e258299c.jpg',
 'rose\\1d04417b3c05c8d2b41d3e7e9b827363.jpg',
 'rose\\1d2e09b3f0506fe3677dbb6825252bb9.jpg',
 'rose\\1d715ae0126de9acad6d2a41851a90c1.jpg',
 'sunflower\\4a09008bbad07e2796b7fd697fa36abd.jpg',
 'sunflower\\4ab53e59a94c4c1d989dd770aeeb67df.jpg',
 'sunflower\\4ada877ac9c6d9f0145a4cdb17c3d775.jpg',
 'sunflower\\4ce82c83b59ad1060120baff169dd724.jpg',
 'sunflower\\4d747cb7808bee432edf8e3c3f6e3d89.jpg',
 'tulip\\0c4ac728ef1f6de1e7c79e541f72a0c7.jpg',
 'tulip\\0c74a7661e8f19b66b6189692b2b9168.jpg',
 'tulip\\0f03fbd73eb0c123e0aa6de63e313b7d.jpg',
 'tulip\\1b1cc31d2d6a6720fd545d8c32345441.jpg']

In [21]:
file_name[0]

'daisy\\1b212edb4f26f7f9008bbf761568c751.jpg'

In [23]:
img = load_img(os.path.join(IMG_DIR,file_name[0]))
img_array = img_to_array(img)
img_array =img_array.reshape((1,)+ img_array.shape)

In [29]:
i = 0 
for batch in aug_gen.flow(img_array, batch_size=1, 
                          save_to_dir=IMG_DIR,save_prefix='daisy',
                          save_format='jpeg'):
    i+=1
    if i> 3:
        break

In [32]:
x[1]

ValueError: Asked to retrieve element 1, but the Sequence has length 1

In [5]:
# Load training set
train = train_datagen.flow_from_directory(TRAIN_DIR,
                                          target_size=(64,64), 
                                          batch_size=32, 
                                          class_mode='categorical',
                                         )
total_samples = train.n
# Load testing set
test = test_datagen.flow_from_directory(TEST_DIR,
                                        target_size=(64,64),
                                        batch_size=1, 
                                        class_mode=None,
                                        shuffle=False)
# train = keras.utils.to_categorical(train, 5)
# test = keras.utils.to_categorical(test,5)

Found 2823 images belonging to 5 classes.
Found 2000 images belonging to 1 classes.


In [12]:
train.index_generator

<generator object Iterator._flow_index at 0x000001B303B28DB0>

In [33]:
clf = Sequential()
clf.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=(64,64,3)))
clf.add(Activation('relu'))
clf.add(Dense(num_classes))
clf.add(Activation('softmax'))
clf.summary()
clf.compile(optimizer=RMSprop(), loss='categorical_crossentropy', 
            metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 64, 64, 5)         325       
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 5)         0         
Total params: 2,117
Trainable params: 2,117
Non-trainable params: 0
_________________________________________________________________


In [36]:
aug_gen.fit(x)
history = clf.fit_generator(aug_gen.flow(x),
                              epochs=epochs,
                              steps_per_epoch=int(len(x)/32),
                              verbose=1,
                              )

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
clf.fit_generator(train,
                  steps_per_epoch = int(total_samples/batch_size),
                  epochs = 50,
                  verbose=1,
                  shuffle=True
                  )

In [171]:
# Transfer learning using resnet
model_resnet = resnet(input_shape=(64,64,3), num_classes=num_classes)
model_resnet.summary()
model_resnet.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
model_resnet.fit_generator(train,
                  steps_per_epoch = int(total_samples/batch_size),
                  epochs = 25,
                  verbose=1,
                  shuffle=True
                  )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_135 (Conv2D)             (None, 64, 64, 16)   448         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 64, 64, 16)   64          conv2d_135[0][0]                 
__________________________________________________________________________________________________
activation_120 (Activation)     (None, 64, 64, 16)   0           batch_normalization_113[0][0]    
__________________________________________________________________________________________________
conv2d_136

Epoch 1/25
 5/88 [>.............................] - ETA: 18:44 - loss: 2.2229 - acc: 0.2750

KeyboardInterrupt: 

In [123]:
test.reset()
pred = model_resnet.predict_generator(test, steps=len(test),verbose=1)
predicted_class_indices=np.argmax(pred, axis=1)



2000/2000 [==============================] - 134s 67ms/step


In [158]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("\\")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission1.csv", index=False)

In [160]:
results.to_csv("./submission1.csv", index=False)

In [143]:
front, back= filenames.split('\\')
back

AttributeError: 'list' object has no attribute 'split'

In [173]:
clf.fit_generator(train,
                  steps_per_epoch = int(total_samples/batch_size),
                  epochs = 50,
                  verbose=1,
                  shuffle=True
                  )

Epoch 1/50
88/88 [==============================] - 89s 1s/step - loss: 1.5526 - acc: 0.3261
Epoch 2/50
88/88 [==============================] - 91s 1s/step - loss: 1.2736 - acc: 0.4534
Epoch 3/50
88/88 [==============================] - 85s 971ms/step - loss: 1.1690 - acc: 0.5303
Epoch 4/50
88/88 [==============================] - 86s 975ms/step - loss: 1.1042 - acc: 0.5664
Epoch 5/50
88/88 [==============================] - 87s 992ms/step - loss: 1.0144 - acc: 0.6026
Epoch 6/50
88/88 [==============================] - 86s 973ms/step - loss: 1.0234 - acc: 0.6079
Epoch 7/50
88/88 [==============================] - 86s 974ms/step - loss: 0.9508 - acc: 0.6397
Epoch 8/50
88/88 [==============================] - 85s 971ms/step - loss: 0.9394 - acc: 0.6546
Epoch 9/50
88/88 [==============================] - 86s 979ms/step - loss: 0.9315 - acc: 0.6446
Epoch 10/50
88/88 [==============================] - 88s 1s/step - loss: 0.9080 - acc: 0.6486
Epoch 11/50
88/88 [=============================

KeyboardInterrupt: 

In [107]:
labels = (train.class_indices)
labels = dict((n,l) for l,n in labels.items())
labels

{0: 'daisy', 1: 'dandelion', 2: 'rose', 3: 'sunflower', 4: 'tulip'}

In [174]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
clf.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 


Saved model to disk


In [176]:
test.reset()
pred = clf.predict_generator(test, steps=len(test),verbose=1)
predicted_class_indices=np.argmax(pred, axis=1)


2000/2000 [==============================] - 45s 22ms/step


In [177]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("\\")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission2.csv", index=False)

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [83]:
def ResNet50(input_shape = (64, 64, 3), classes = 5):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = keras.Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128,128,512],stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128,128,512], stage=3, block='b')
    X = identity_block(X, 3, [128,128,512], stage=3, block='c')
    X = identity_block(X, 3, [128,128,512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256,256,1024],stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256,256,1024], stage=4, block='b')
    X = identity_block(X, 3, [256,256,1024], stage=4, block='c')
    X = identity_block(X, 3, [256,256,1024], stage=4, block='d')
    X = identity_block(X, 3, [256,256,1024], stage=4, block='e')
    X = identity_block(X, 3, [256,256,1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters=[512,512,2048],stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512,512,2048], stage = 5, block ='b')
    X = identity_block(X, 3, [512,512,2048], stage = 5, block ='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), strides=None, name='avg_pool')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model    

In [84]:
res_model = ResNet50()
res_model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
res_model.fit_generator(train,
                  steps_per_epoch = int(len(train)/32),
                  epochs = 25,
                  verbose=1,
                  shuffle=True
                  )

NameError: name 'convolutional_block' is not defined

In [20]:
def img_combine(img, ncols=8, size=1, path=False):
    from math import ceil
    import matplotlib.pyplot as plt
    import numpy as np
    nimg = len(img)
    nrows = int(ceil(nimg/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(ncols*size,nrows*size))
    if nrows == 0:
        return
    elif ncols == 1:
        for r, ax in zip(np.arange(nrows), axes):
            nth=r
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                
            ax.set_axis_off()
    elif nrows == 1:
        for c, ax in zip(np.arange(ncols), axes):
            nth=c
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
            ax.set_axis_off()
    else:
        for r, row in zip(np.arange(nrows), axes):
            for c, ax in zip(np.arange(ncols), row):
                nth=r*ncols+c
                if nth < nimg:
                    ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                ax.set_axis_off()
    plt.show()

In [58]:
score = clf.predict_classes(test)
# predict(test, verbose=0)


AttributeError: 'DirectoryIterator' object has no attribute 'ndim'

In [18]:
ROOT_DIR

'C:\\Users\\cptien\\Desktop\\data'